In [55]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ConfigSpace import Categorical, Configuration, ConfigurationSpace, Float, Integer
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from smac import HyperparameterOptimizationFacade, Scenario

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.alpha"] = 0.2
plt.rcParams["grid.color"] = "#cccccc"
plt.rcParams["axes.xmargin"] = 0

PHASE1_DIR = Path(".")
PROCESSED_DIR = PHASE1_DIR / "processed"

In [56]:
evaluations_df = pd.read_parquet(PROCESSED_DIR / "evaluations.parquet")
solvers_df = pd.read_parquet(PROCESSED_DIR / "solvers.parquet")
instances_df = pd.read_parquet(PROCESSED_DIR / "instances.parquet")

df = pd.merge(evaluations_df, solvers_df, left_on="solver_id", right_on="id").drop(columns=["id"])
df = pd.merge(df, instances_df, left_on="instance_id", right_on="id").drop(columns=["id"])
df

,solver_id,instance_id,generator,cost,log_cost,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,...,mst_dists_span,mst_dists_coef_of_var,mst_dists_sum,nnds_min,nnds_median,nnds_mean,nnds_max,nnds_sd,nnds_span,nnds_coef_of_var
0,1251473931473582278,TSP/TRAIN/cluster_netgen/000.tsp,cluster_netgen,4.74,1.747459,0.95,1.0,0.0,0.0,0.2,...,0.488488,1.790135,0.000225,0.000715,0.011143,0.014421,0.094965,0.012746,0.094250,0.883825
1,2289112522627003788,TSP/TRAIN/cluster_netgen/000.tsp,cluster_netgen,0.02,0.019803,0.15,0.0,1.0,1.0,0.4,...,0.488488,1.790135,0.000225,0.000715,0.011143,0.014421,0.094965,0.012746,0.094250,0.883825
2,960932965817811220,TSP/TRAIN/cluster_netgen/000.tsp,cluster_netgen,3.72,1.551809,0.20,0.0,1.0,2.0,0.9,...,0.488488,1.790135,0.000225,0.000715,0.011143,0.014421,0.094965,0.012746,0.094250,0.883825
3,39012066323493184,TSP/TRAIN/cluster_netgen/000.tsp,cluster_netgen,1.52,0.924259,0.60,1.0,1.0,2.0,0.7,...,0.488488,1.790135,0.000225,0.000715,0.011143,0.014421,0.094965,0.012746,0.094250,0.883825
4,494182449327999052,TSP/TRAIN/cluster_netgen/000.tsp,cluster_netgen,84.44,4.447814,0.90,1.0,1.0,3.0,0.3,...,0.488488,1.790135,0.000225,0.000715,0.011143,0.014421,0.094965,0.012746,0.094250,0.883825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1286196444435323941,TSP/TRAIN/grid/019.tsp,grid,208.32,5.343864,0.45,0.0,0.0,0.0,0.2,...,0.093952,0.462330,0.000294,0.001335,0.029728,0.032519,0.095287,0.017339,0.093952,0.533201
99996,1435531534300921454,TSP/TRAIN/grid/019.tsp,grid,300.00,5.707110,0.20,1.0,0.0,3.0,0.6,...,0.093952,0.462330,0.000294,0.001335,0.029728,0.032519,0.095287,0.017339,0.093952,0.533201
99997,27607668447685341,TSP/TRAIN/grid/019.tsp,grid,3.21,1.437463,0.95,1.0,1.0,3.0,0.9,...,0.093952,0.462330,0.000294,0.001335,0.029728,0.032519,0.095287,0.017339,0.093952,0.533201
99998,2245205590089179674,TSP/TRAIN/grid/019.tsp,grid,17.23,2.903069,0.65,0.0,0.0,1.0,0.5,...,0.093952,0.462330,0.000294,0.001335,0.029728,0.032519,0.095287,0.017339,0.093952,0.533201


In [57]:
# the way of evaluation
# hyperparameter tuning
# models
# results

In [58]:
def get_n_splits(df, n, instance_number, solver_number, random_state=0):
    """
    Generate indices to split data into training and test sets.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing solver evaluation data
    n : int
        Number of splits
    instance_number : int
        Number of instances to select
    solver_number : int
        Number of solver configurations to select
    random_state : int, default=0
        Random state for reproducibility
        
    Yields:
    -------
    train_idx : numpy.array
        Indices of training set
    test_idx : numpy.array
        Indices of test set
        
    Notes:
    ------
    1. Randomly selects solver_number solvers and instance_number instances from df
    2. Ensures that all evaluations from one solver are either in train or test set
    """
    rng = np.random.default_rng(random_state)
    
    if solver_number % n != 0:
        raise ValueError(f"solver_number ({solver_number}) must be divisible by n ({n})")
    
    all_solver_ids = df['solver_id'].unique()
    all_instance_ids = df['instance_id'].unique()
    
    if len(all_solver_ids) < solver_number:
        raise ValueError(f"Not enough solvers in df ({len(all_solver_ids)}) to select {solver_number}")
    
    if len(all_instance_ids) < instance_number:
        raise ValueError(f"Not enough instances in df ({len(all_instance_ids)}) to select {instance_number}")
    
    selected_solver_ids = rng.choice(all_solver_ids, size=solver_number, replace=False)
    selected_instance_ids = rng.choice(all_instance_ids, size=instance_number, replace=False)
    
    subset_df = df[
        df['solver_id'].isin(selected_solver_ids) & 
        df['instance_id'].isin(selected_instance_ids)
    ]
    
    expected_rows = solver_number * instance_number
    if len(subset_df) != expected_rows:
        raise ValueError(f"Incomplete data: Found {len(subset_df)} rows instead of expected {expected_rows}")
    
    solvers_per_fold = solver_number // n
    shuffled_solver_ids = rng.permutation(selected_solver_ids)
    
    for i in range(n):
        test_solvers = shuffled_solver_ids[i * solvers_per_fold:(i + 1) * solvers_per_fold]
        
        test_idx = subset_df[subset_df['solver_id'].isin(test_solvers)].index.values
        train_idx = subset_df[~subset_df['solver_id'].isin(test_solvers)].index.values
        
        total_samples = len(train_idx) + len(test_idx)
        if total_samples != expected_rows:
            raise ValueError(f"Total samples ({total_samples}) doesn't match expected number ({expected_rows})")
        
        yield train_idx, test_idx

In [59]:
def evaluate_model_with_cross_validation(
    df, 
    model, 
    n=5, 
    instance_number=10, 
    solver_number=1000, 
    random_state=0,
):
    """
    Evaluate a model using n-fold cross-validation, recording performance metrics.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing solver evaluation data
    model : estimator object
        The model to be trained and evaluated
    n : int, default=5
        Number of splits
    instance_number : int, default=10
        Number of instances to select
    solver_number : int, default=1000
        Number of solver configurations to select
    random_state : int, default=0
        Random state for reproducibility

    Returns:
    --------
    dict
        Dictionary with keys:
        - 'model': the model name or object
        - 'rmse': average root mean squared error across splits
        - 'cc': average Pearson correlation coefficient across splits
        - 'r2': average R² score across splits
    """
    result = {"model": model, "rmse_values": [], "cc_values": [], "r2_values": []}

    for train_idx, test_idx in get_n_splits(
        df,
        n=n,
        instance_number=instance_number,
        solver_number=solver_number,
        random_state=random_state,
    ):
        df_train = df.loc[train_idx]
        X_train = df_train.drop(columns=["solver_id", "instance_id", "generator", "cost", "log_cost"])
        y_train = df_train["log_cost"]

        df_test = df.loc[test_idx]
        X_test = df_test.drop(columns=["solver_id", "instance_id", "generator", "cost", "log_cost"])
        y_test = df_test["log_cost"]

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        cc, _ = pearsonr(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        result["rmse_values"].append(rmse)
        result["cc_values"].append(cc)
        result["r2_values"].append(r2)

    result["rmse"] = np.mean(result["rmse_values"])
    result["cc"] = np.mean(result["cc_values"])
    result["r2"] = np.mean(result["r2_values"])

    return result

In [101]:
def optimize_hyperparameters(
    model_cls,
    configspace,
    df,
    n_trials=30,
    n=3,
    instance_number=10,
    solver_number=300,
    random_state=0,
):
    def train(config: Configuration, seed: int = 0) -> float:
        model = model_cls(**config, n_jobs=-1, random_state=seed)
        result = evaluate_model_with_cross_validation(
            df,
            model,
            n=n,
            instance_number=instance_number,
            solver_number=solver_number,
            random_state=seed,
        )
        return 1 - result["r2"]

    scenario = Scenario(configspace, deterministic=True, n_trials=n_trials, seed=random_state)
    smac = HyperparameterOptimizationFacade(scenario, train, overwrite=True)
    return smac.optimize()

configspace = ConfigurationSpace(
    seed=0,
    space=[
        Integer(name="max_depth", bounds=(2, 16), default=16),
    ],
)

optimize_hyperparameters(
    model_cls=RandomForestRegressor,
    configspace=configspace,
    df=df,
    n_trials=10,
    n=3,
    instance_number=10,
    solver_number=300,
    random_state=0,
)

[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 10 to 2 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 2 initial design configurations and 0 additional configurations.
[INFO][abstract_intensifier.py:306] Using only one seed for deterministic scenario.
[INFO][abstract_intensifier.py:516] Added config 0a3041 as new incumbent because there are no incumbents yet.
[INFO][abstract_intensifier.py:595] Added config 3a84fe and rejected config 0a3041 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:595] Added config e05058 and rejected config 3a84fe as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:595] Added config 4267b2 and rejected config e05058 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:328] Configuration budget is exhausted:
[INFO][smbo.py:329] --- Remaining wallclock time: in

Configuration(values={
  'max_depth': 16,
})

In [37]:
model = RandomForestRegressor(n_jobs=-1)
model

RandomForestRegressor(n_jobs=-1)

In [42]:
evaluate_model_with_cross_validation(df, model, n=2, instance_number=10, solver_number=1000, random_state=0)

500 10 500 10
500 10 500 10


{'model': RandomForestRegressor(n_jobs=-1),
 'rmse_values': [1.2519633057504824, 1.2425560626805565],
 'cc_values': [0.8139885338947487, 0.8200966532922144],
 'r2_values': [0.66082274590915, 0.6658368501861383],
 'rmse': 1.2472596842155195,
 'cc': 0.8170425935934815,
 'r2': 0.6633297980476441}